## Testing trajectory extractions

### Scenes first

In [ ]:
# Specify trajectory by scene -> category -> index
# obs_dict_list, done_list = \
#     get_traj_data_by_scene_category_trajIdx(scenes_cats_trajs_dict, "D7N2EKCX4Sj", "chair", 0, tensorize=True)

# Specify trajectories by scene, then category
# obs_dict_list, done_list = \
#     get_traj_data_by_scene_category(scenes_cats_trajs_dict, "D7N2EKCX4Sj", "chair", tensorize=True)

# All trajectories by scene, for all categories
# obs_dict_list, done_list = \
#     get_traj_data_by_scene(scenes_cats_trajs_dict, "D7N2EKCX4Sj", tensorize=True)

# region: D7N2EKCX4Sj, all categories, all trajectories
# D7N2EKCX4Sj_allcats_alltrajs_obs_dict_list, \
# D7N2EKCX4Sj_allcats_alltrajs_done_list, \
# D7N2EKCX4Sj_allcats_alltrajs_target_scene_idx_list, \
# D7N2EKCX4Sj_allcats_alltrajs_target_category_idx_list = \
#         get_traj_data_by_scene(scenes_cats_trajs_dict, "D7N2EKCX4Sj", tensorize=True)

# # Special sections will override "obs_dict_list" and "done_list"
# obs_dict_list, done_list = \
#         D7N2EKCX4Sj_allcats_alltrajs_obs_dict_list, \
#         D7N2EKCX4Sj_allcats_alltrajs_done_list

# # The following three should all be equal: one target for scene / category for each step
# len(D7N2EKCX4Sj_allcats_alltrajs_obs_dict_list), len(D7N2EKCX4Sj_allcats_alltrajs_target_scene_idx_list), len(D7N2EKCX4Sj_allcats_alltrajs_target_category_idx_list)
# endregion: D7N2EKCX4Sj, all categories, all trajectories


# print(f"done_list shape: {np.shape(done_list)}")
# print(len(obs_dict_list))

# print(np.array(done_list).astype(np.uint8))
# for k, v in obs_dict_list.items():
#     print(f"{k}: {np.shape(v)}")

## Process agent's features (revisited)

In [ ]:
# ## Helper for cleaning up and preparing the recorded intermediate features
# def process_analysis_feats_raw(raw_dict):
#     result_dict = {}

#     for k, v in raw_dict.items():
#         if isinstance(v[0], th.Tensor):
#             new_v = th.cat(v, dim=0)
#         elif isinstance(v[0], tuple):
#             new_v = None # TODO
#             n_elements = len(v[0])
#             elements = [[] for _ in range(n_elements)]
#             for j in range(n_elements):
#                 for i in range(len(v)):
#                     elements[j].append(v[i][j])
            
#             new_v = [th.cat(vv, dim=0).cpu().numpy() for vv in elements]
#         else:
#             raise Exception(f"Unhandled type: {v[0].__class__}")
    
#         result_dict[k] = new_v
    
#     return result_dict

In [ ]:
# # For the collected observation samples, record the intermediate features
# # for each agent varaint investigated

# AGENT_FEATURES__RAW = {k: {} for k in MODEL_VARIANTS_TO_AGENTMODEL.keys()}
# AGENT_RNN_HIDDEN_STATE = {}
# for agent_variant, agent_model in MODEL_VARIANTS_TO_AGENTMODEL.items():
#     if agent_variant.__contains__("gru"):
#         AGENT_RNN_HIDDEN_STATE[agent_variant] = th.zeros((1, args.num_envs, args.hidden_size), device=dev)
#     elif agent_variant.__contains__("pgwt"):
#         AGENT_RNN_HIDDEN_STATE[agent_variant] = agent_model.state_encoder.latents.clone()

# # TODO: Add tqdm support ?
# # for t, (obs_th, done_th) in enumerate(zip(obs_th_list, done_th_list)):
# for t, (obs_th, done_th) in enumerate(zip(obs_dict_list, done_list)):
#     # TODO: if multiple episodes are cated together, need to make sure that the done_th_list reflects that
#     # other the hidden latents will not be reset

#     # Feeding the same sequence of observatiosn to each type of agent
#     masks = 1. - done_th[:, None]
#     with th.no_grad():
#         for agent_variant, agent_model in MODEL_VARIANTS_TO_AGENTMODEL.items():
#             # NOTE: rnn_hidden_state or pgwt's latent are already collected in "state_encoder" field in _features
#             _, _, _, _, _, _, AGENT_RNN_HIDDEN_STATE[agent_variant] = \
#                 agent_model.act(obs_th, AGENT_RNN_HIDDEN_STATE[agent_variant], masks)

#             # Collecting intermediate layers results
#             for k, v in agent_model._features.items():
#                 if k not in list(AGENT_FEATURES__RAW[agent_variant].keys()):
#                     AGENT_FEATURES__RAW[agent_variant][k] = []
#                 AGENT_FEATURES__RAW[agent_variant][k].append(v)

# # After recording the intermediate layers features, process them to handle the 
# # various shape depending on the layers: for example MHA has different stored features
# # shape than GRU network, and so on.

# AGENT_FEATURES = {} # { agent_variant -> agent_feature_dict }
# for k, v in AGENT_FEATURES__RAW.items():
#     AGENT_FEATURES[k] = process_analysis_feats_raw(v)

In [ ]:
# Systematic plotting for each shared intermediate layers
all_layers = []
for v in AGENT_FEATURES.values():
    all_layers.extend(v.keys())
all_layers = set(all_layers); all_layers

# Extract shared layers, and other groups of layers of interest
shared_layers = []
visual_encoder_layernames = []
audio_encoder_layernames = []
for x in all_layers:
    x_found_in_all = True
    for v in AGENT_FEATURES.values():
        if x not in v.keys():
            x_found_in_all = False
            break
    
    if x_found_in_all:
        shared_layers.append(x)
    
    if x.startswith("visual_encoder"):
        visual_encoder_layernames.append(x)
    if x.startswith("audio_encoder"):
        audio_encoder_layernames.append(x)

In [ ]:
# Cache cat -> scene -> traj -> ablation type -> agent variant -> features data into independent files
# to facilitate read into memory later
# Keeping everything in memroy takes just too much
CACHE_DIRNAME = "cats_scenes_trajs_C_6_M_5_N_5__2023_06_01_10_41__ablation_agents_features__cache"

# NOTE: Uncomment for cache to disk
# if not os.path.exists(CACHE_DIRNAME):
#     os.mkdir(CACHE_DIRNAME)

# for catname, scene_trajs_ablations_agentvariant_features_dict in CAT_SCENE_TRAJS_ABLATION_AGENTVARIANT_FEATURES.items():
#     cat_dirname = f"{CACHE_DIRNAME}/{catname}"
#     if not os.path.exists(cat_dirname):
#         os.mkdir(cat_dirname)
    
#     for scene, trajs_ablations_agentvariant_features_dict in scene_trajs_ablations_agentvariant_features_dict.items():
#         scene_dirname = f"{cat_dirname}/{scene}"
#         if not os.path.exists(scene_dirname):
#             os.mkdir(scene_dirname)
        
#         for traj_idx, ablations_agentvariant_features_dict in trajs_ablations_agentvariant_features_dict.items():
#             traj_dirname = f"{scene_dirname}/{traj_idx}"
#             if not os.path.exists(traj_dirname):
#                 os.mkdir(traj_dirname)
            
#             for ablation_type, agentvariant_features_dict in ablations_agentvariant_features_dict.items():
#                 if ablation_type in ["target_category_idx_list", "target_scene_idx_list"]:
#                     # Just saves those two fields as metadata
#                     ablation_type_filename = f"{traj_dirname}/{ablation_type}.bz2"

#                     # TODO: control on the existence of the file ?
#                     with open(ablation_type_filename, "wb") as f:
#                         # In this case, "agentvariant_features_dict" is just a list
#                         cpkl.dump(agentvariant_features_dict, f)
                    
#                     # This will skip metadata about the trajectory, namely the labels for probes
#                     continue
                
#                 ablation_dirname = f"{traj_dirname}/{ablation_type}"
#                 if not os.path.exists(ablation_dirname):
#                     os.mkdir(ablation_dirname)
                
#                 for agent_variant, features_dict in agentvariant_features_dict.items():
#                     agent_variant_dirname = f"{ablation_dirname}/{agent_variant}"
#                     if not os.path.exists(agent_variant_dirname):
#                         os.mkdir(agent_variant_dirname)
                    
#                     feature_dict_filename = f"{agent_variant_dirname}/features.bz2"

#                     if not os.path.exists(feature_dict_filename):
#                         with open(feature_dict_filename, "wb") as f:
#                             cpkl.dump(features_dict, f)
